In [ ]:
import numpy as np
from collections import defaultdict
import json
import copy

In [ ]:
# average minimum contract is this overall
r_lvl = 44.13556768970062

# wins over replacement based on ovr
def value_predict(ovr):
    xpm = np.array([6.411, -61.762, 0.094, 6.89])
    return xpm[3]*np.tanh(xpm[2]*(ovr+xpm[1]))+xpm[0]

# average progs
def get_progs(age):
    progs_vec = np.array([  4.00191771, -21.03934289,  -0.11999193,   8.9637918 ])
    return progs_vec[3]*np.tanh(progs_vec[2]*(age+progs_vec[1]))+progs_vec[0]

# expected mov weight for salary cap space, , weight for players, 
weights = {
0 : ( [-2.971, 0.0, 0.497] ),
1 : ( [-1.431, 2.809, 0.375] ),
2 : ( [-0.633, 1.396, 0.252] ),
}

# draft value in two different ways
def draft_value(ovr,pot,age):
    vec = [3.386540731619821, 0.093735237239775, 0.05825623604611395, -0.669579845047606]
    return np.tanh(vec[0] + vec[1]*ovr + vec[2]*pot + vec[3]*age)*0.5 + 0.5

def draft_value(draft_pick):
    draftP = np.array([0.21335084, 0.21841752, 0.64062662])
    return draftP[1]*np.exp(-draftP[0]*draft_pick**draftP[2])

# team value
def team_mov(team_ovrs):
    tr_x = [-0.12892717787376945, 0.2876446335881824, -97.97478635988327]
    team_ovrs = sorted(team_ovrs)[::-1]
    return sum([np.exp(tr_x[0]*idx)*tr_x[1]*ovr for idx,ovr in enumerate(team_ovrs[:10])]) + tr_x[2]

def avg_ovr(draft_pick):
    dp_ovr_x = np.array([-0.14244607, 38.97350078, 13.02392059])
    return np.exp(dp_ovr_x[0]*draft_pick**0.5) *dp_ovr_x[1]+dp_ovr_x[2]

# salary multiplier
sA = 9.18

# chance of making first team based on OVR
def make_first_team(ovr):
    fTA,fTB = 0.4937998843623995, -35.85000013688516
    return 1.0/(1+np.exp(-ovr*fTA -fTB))

In [ ]:
age_shift_int = defaultdict(int)
for draft_age in range(17,27):
    for age in range(draft_age,27):
        total = 0
        seen = 0
        for i in range(draft_age,27):
            prog = get_progs(i)
            if prog > 0:
                total += prog
        for i in range(draft_age,age):
            prog = get_progs(i)
            if prog > 0:
                seen += prog
        if total > 0:
            frac = 1.0-seen/total
            if frac > 0:
                age_shift_int[(draft_age,age)] = frac
age_shift_int

In [ ]:
team = { 'p' : [(22,66,22000,0,19,50,70,4),(22,66,22000,1,19,50,70,1),(22,66,22000,2,19,50,70,2),(22,66,22000,3,19,50,70,3)], #(age,ovr,salary,years_left,draft_age,draft_ovr,draft_pot,draft_pick)
         'd' : [(1, 0, 8.6),(1, 0, -5.35),(2,0,3),(1,1,3),(2,1,3),(1,2,3),(2,2,3),(1,3,3),(2,3,3),(1,4,3),(2,4,3)] #(round,years_left,team_MOV)
       }
YEARS_TO_MODEL = 3
DISCOUNT = 1.0 # how much of a hometown discount do players get
                # otherwise bad teams give up good players in trades too easy
                # because they think they'll want an absurd contract & prefer some value

def eval_state(pars,tss,sCap,minS):
    pred_win_p = []
    for i in range(YEARS_TO_MODEL):
        play = [p for p in pars[i] if p >= r_lvl]
        lp = len(play)
        if lp < 10:
            play= play + (10-lp)*[r_lvl]
        play_s = team_mov(play)

        cap_hit = tss[i] + (10-lp)*minS 

        diff = (sCap-cap_hit)/sCap
        cap_space = np.maximum(diff,0.1*diff)
        x = weights.get(i,weights[2])

        
        ppow = x[0] + x[1]*cap_space + x[2]*play_s
        win_p = 1.0/(1+np.exp(-ppow))
        
        pred_win_p.append(win_p)

    # discount factor for the future, more uncertainty, less sure reward
    #value = [wp*(0.9**(i)) for i,wp in enumerate(win_p)]
    return pred_win_p


def get_team_value(team,sCap=90000,minS=750,teamNum=30):
    # turn mov into draft pick and future mov
    m2pos = lambda x: int(round(np.clip( (teamNum-1)/(1+np.exp(0.0009 -  0.3430*(x))),0,teamNum-1)))
    m2next = lambda year,mov: [1,0.65,0.44,0.3,0.2,0.15,0.15,0.11,0.09,0.08][year]*mov 

    # turn draft picks into specific predictions
    draft_picks = [ (yr,m2pos(m2next(yr,mov))+teamNum*(rnd-1)) for rnd,yr,mov in team['d']]
    
    pars = defaultdict(list) # player value
    tss = defaultdict(int)  # contracts
    dpars = [] # draft value
    unique = [] # unique value
    contract_val = [] # extra years of contract val
    
    # analyze existing contracts
    for age,ovr,con,yrl,draft_age,draft_ovr,draft_pot,draft_pick in team['p']:
        ovr2  = ovr
        povrs = [ovr2]
        
        # compute aging value
        for i in range(max(YEARS_TO_MODEL,yrl)):
            ovr2+=get_progs(age+i)
            povrs.append(ovr2)
        all_nba = [make_first_team(ovr2) for ovr2 in povrs]
        unique.append(max(all_nba))
        
        # this is pretty good +/-
        pmovs = [value_predict(ovr2) for ovr2 in povrs]

        # estimate contract value
        ccont = [(sCap/3)*min(1,pmov/sA) for pmov in pmovs]
        
        # how much value do we get each contract year
        cvals = [c-con for c in ccont]

        # add existing contract
        for i in range(yrl+1):
            pars[i].append(povrs[i])
            tss[i] += con
            
        # extend contract (at fair price)
        if yrl+1 < YEARS_TO_MODEL:
            prev_val = eval_state(pars,tss,sCap,minS)

            tss2 = copy.deepcopy(tss)
            pars2 = copy.deepcopy(pars)

            for i in range(yrl+1,YEARS_TO_MODEL):
                pars2[i].append(povrs[i])
                tss2[i] += max(DISCOUNT*ccont[i],minS)
                
            # only if extending is the right thing
            if eval_state(pars2,tss2,sCap,minS) > prev_val:
                pars = pars2
                tss = tss2
        
        # leftover contract value into extra term
        contract_val.append(sum(cvals[YEARS_TO_MODEL:]))
    
    # compute draft pick value
    for yr,p in draft_picks:
        dpars.append((0.95**yr)*draft_value(p))
    
    # add draft picks to roster
    # make them zero cost
    for yr,p in draft_picks:
        if yr+1 < YEARS_TO_MODEL:
            ovr = avg_ovr(p)
            for i in range(yr+1,YEARS_TO_MODEL):
                ovr = ovr + get_progs(20-(yr+1)+i)
                pars[i].append(ovr)
    
    # add young player value into long-term estimate
    for age,ovr,con,yrl,draft_age,draft_ovr,draft_pot,draft_pick in team['p']:
        # for real_rosters draft ratings aren't available!
        # dpars.append(age_shift_int.get((draft_age,age),0)*winp_draft(draft_ovr,draft_pot,draft_age))
        dpars.append(age_shift_int.get((draft_age,age),0)* draft_value(draft_pick))

    value = eval_state(pars,tss,sCap,minS)
    #print(contract_val)
    print([round(_,2) for _ in [sum(value) , sum(dpars) , sum(unique) , 0.1*sum(contract_val)/maxS]])
    return sum(value) + sum(dpars) + sum(unique) + 0.1*sum(contract_val)/maxS
get_team_value(team,90000,750)

In [ ]:
data = json.load(open('21_playoffs.json','rt',encoding='utf-8-sig')) #real_2020 #war2
gA =  data['gameAttributes']

In [ ]:
team_players = defaultdict(list)
team_names = {}
season = gA['season']
sCap = gA['salaryCap']
minS = gA['minContract']
maxS = gA['maxContract']

team_injuries = defaultdict(list)
for p in data['players']:
    #draft_age,draft_ovr,draft_pot
    if p['tid'] >= 0:
        ovr = p['ratings'][-1]['ovr']
        age = season - p['born']['year']
        salary = p['contract']['amount']
        years_left = p['contract']['exp']-season
        dp = p['draft']['pick'] + 30 * (p['draft']['round']-1)
        if dp < 0:
            dp = 61
        team_players[p['tid']].append((age,ovr,salary,years_left,p['draft']['year']-p['born']['year'],p['draft']['ovr'],p['draft']['pot'],dp))
        team_injuries[p['tid']].append(p['injury']['gamesRemaining'])

In [ ]:
team_movs = defaultdict(float)

for t in data['teams']:
    tid = t['tid']
    print(t['abbrev'])
    team_names[t['tid']] = t['abbrev']
    for ts in t['stats']:
        if ts['playoffs']:
            continue
        current_mov = 0
        gp = ts['gp']+1e-9
        gl = 82-gp+1e-9
        if season == ts['season'] and not ts['playoffs'] and ts['gp']>0:
            mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
            current_mov = mov
        estimated_mov = team_mov([_[1] for _ in team_players[tid]])
        team_movs[tid] = (gp/82)*current_mov + (gl/82)*estimated_mov    

In [ ]:
team_picks = defaultdict(list)
for d in data['draftPicks']:
    mov = team_movs[d['originalTid']]
    tid = d['tid']
    rnd = d['round']
    yl = d['season']-season
    team_picks[tid].append((rnd,yl,mov))

In [ ]:
teams_vals = []
for i in range(len(data['teams'])):
    print(team_names[i])
    val = get_team_value({'mov':team_movs[i],'p':team_players[i],'d':team_picks[i]},sCap,minS)
    teams_vals.append((val,team_names[i]))

In [ ]:
for v,t in sorted(teams_vals)[::-1]:
    print(round(100*v,2),t)

In [ ]:
import random
deals = []
for i in range(100):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1], 'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2], 'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    pn = np.random.randint(len(team_picks[t1]))
    picks = [_ for _ in team_picks[t1]]
    pick = picks[pn]        
    del picks[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':team_players[t1] + [player],'d':picks},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p,'d':team_picks[t2] + [pick]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = min((t1v-t1vo), (t2v-t2vo))
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
sorted(deals,reverse=True)

In [ ]:
import random
deals = []
for i in range(100):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':team_picks[t2]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = (t1v-t1vo) + (t2v-t2vo)
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
for d in sorted(deals,reverse=True):
    if d[4][2]/d[2][2] < 1.25 and d[4][2]/d[2][2] > 1/1.25:
        print(d)

In [ ]:
import random
deals = set()
for i in range(100):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        if not (pick[2]/player[2] < 1.25 and pick[2]/player[2] > 1/1.25):
            continue
        for p2 in range(len(team_picks[t2])):
            local_picks = [_ for _ in team_picks[t2]]
            pick2 = local_picks[p2]       
            del local_picks[p2]
            t1v1 = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)

            t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1] + [pick2]},sCap,minS)
            t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':local_picks },sCap,minS)
            if t1v1 < t1vo and t1v > t1vo and t2v > t2vo:
                v1 = (t1v-t1vo)
                v2 = (t2v-t2vo)
                val = min( v1 , v2)
                deals.add((val,v1,v2,team_names[t1],pick,team_names[t2],player,pick2))
                

In [ ]:
sorted(list(deals),reverse=True)

In [ ]:
team_names